In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', None)

import re

import seaborn as sea
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

import sys
sys.path.insert(0, '/data16/marcus/scripts/nanoporePipelineScripts')
from nanoporePipelineCommon import *

print("imports done")

In [ ]:
gtf_parquet_path = "/data16/marcus/genomes/elegansRelease100/Caenorhabditis_elegans.WBcel235.100.gtf.parquet"

gtf_df = pd.read_parquet(gtf_parquet_path)

In [ ]:
gtf_df.query("gene_name == 'ubl-1'").query("feature == 'transcript'")

In [ ]:
import pyranges

bed = pyranges.read_bed("/data16/marcus/genomes/elegansRelease100/Caenorhabditis_elegans.WBcel235.100.bed").df

In [ ]:
bed_df = bed.copy()
bed_df[['transcipt_name', 'gene_type', 'gene_name']] = bed_df.Name.str.split("|", expand=True)
bed_df

In [ ]:
def str_list_to_list_of_ints(row, column):
    if col == 'ItemRGB':
        last_parse_keeper = 3
    else:
        last_parse_keeper = -1
    return list(map(int, row[column].split(',')[:last_parse_keeper]))

for col in ['BlockSizes', 'BlockStarts', 'ItemRGB']:
    try:
        tqdm.pandas(desc=f"Parsing the column: {col}")
        bed_df[col] = bed_df.progress_apply(lambda row: str_list_to_list_of_ints(row, col), axis=1)
    except AttributeError:
        print(f"The {col} already appears to have been parsed!! Ignoring this column!")

In [ ]:
test_df = bed_df.query("gene_name == 'ubl-1'").reset_index(drop=True)
test_df

In [ ]:
fig, ax = plt.subplots()
def row_plot_rectangles(row, axes):
    index = row.name
    transcript_start = row['Start']
    transcript_edit = row['End']
    cds_start = row['ThickStart']
    cds_end = row['ThickEnd']
    exon_starts = row['BlockStarts']
    exon_lengths = row['BlockSizes']
    print(f"\n{row['transcipt_name']}")
    rectangle_patches = []
    for (exon_start, exon_length) in zip(exon_starts, exon_lengths):
        rectangle_patches.append(Rectangle((exon_start, index), exon_length, height=1))
    axes.add_collection(PatchCollection(rectangle_patches))

test_df.progress_apply(lambda row: row_plot_rectangles(row, ax), axis=1)

ax.set_ylim(0,5)
ax.set_xlim(0,1000)
#fig.show()